# Examen Deep Learning

Examen de 3 heures, sur machine, tout documents autorisés. Pour l'anonymisation de votre rendu, veuillez suivre les consignes : 

- Choisissez un numéro aléatoire à 6 chiffres, le plus aléatoire possible..
- Ecrivez ce numéro sur une feuille, ainsi que votre nom et numéro d'étudiant 
- Pliez cette feuille en 4, et donnez la à la fin de l'examen au moment d'émarger  
- Zippez votre fichier notebook, et nommez l'archive avec votre numéro aléatoire 
- Envoyez l'archive via la page : http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/upload/upload.php

<hr/>

Un musée spécialisé en peintures représentant des animaux de la savane a malheureusement perdu son fichier d'inventaire qui regroupait des informations précieuses sur les 2000 oeuvres du musée ! Fort heureusement, l'informaticien toujours prévoyant avait conservé une copie des 2000 photos des oeuvres, et il vient juste de suivre une formation de Deep Learning ...

<center>
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/0.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/1.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/2.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/3.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/4.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/5.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/6.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/7.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/8.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/9.jpg" width="90px" />
</center>

Cet examen porte sur la résolution d'une tâche de classification d'images sur un jeu de données faiblement annoté. Plusieurs solutions sont envisagées  consistant Ã  tirer parti d'annotations de classes équivalentes sur un autre dataset entièrement étiquetté (imagenet). Les deux jeux de données contiennent 2000 images réparties (également) en quatre classes (d'animaux : zèbres, gorilles, léopards, tigres). 1500 images sont utilisées pour l'entrainement, 500 pour l'évaluation. Seul le jeu de données A (imagenet) est complètement annoté, le dataset B (les photos des oeuvres du musée) ne contient que très peu d'annotations.

La partie 1 consiste à entrainer et évaluer les performances d'un modèle CNN fourni sur les deux datasets d'images. Les parties 2 et 3 consistent à mettre en oeuvre deux solutions par *adaptation de domaines* pour tenter d'améliorer les performances de classification sur le dataset faiblement annoté. Vous pouvez consacrer environ une heure à chaque partie, notées également.


In [1]:
try:
    # core libraries
    %tensorflow_version 2.x
    import numpy as np
    import tensorflow as tf
    from tensorflow.keras import layers

    # plot libraries
    import matplotlib.pyplot as plt
except Exception:
    print('Tensorflow version 2 not available')

TensorFlow 2.x selected.


## Téléchargement des données

In [2]:
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetXtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetYtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetXtest.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetYtest.npy
  
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaXtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaYtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaXtest.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaYtest.npy  

--2019-11-19 12:37:20--  http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetXtrain.npy
Resolving stephane.ayache.perso.luminy.univ-amu.fr (stephane.ayache.perso.luminy.univ-amu.fr)... 139.124.69.89
Connecting to stephane.ayache.perso.luminy.univ-amu.fr (stephane.ayache.perso.luminy.univ-amu.fr)|139.124.69.89|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18432128 (18M) [text/plain]
Saving to: ‘imagenetXtrain.npy.1’

imagenetXtrain.npy. 100%[===================>]  17.58M  1.42MB/s    in 29s     

2019-11-19 12:37:49 (624 KB/s) - ‘imagenetXtrain.npy.1’ saved [18432128/18432128]

--2019-11-19 12:37:51--  http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetYtrain.npy
Resolving stephane.ayache.perso.luminy.univ-amu.fr (stephane.ayache.perso.luminy.univ-amu.fr)... 139.124.69.89
Connecting to stephane.ayache.perso.luminy.univ-amu.fr (stephane.ayache.perso.luminy.univ-amu.fr)|139.124.69.89|:80... connected.
HTTP request sent, awaiting respon

### Chargement des données

In [3]:
def get_data(path_x, path_y, n_features):
    x = np.load(path_x) / 255.
    y = tf.keras.utils.to_categorical(np.load(path_y), n_features)
    return x, y

# ---
# load our data
# ---
path = './'
n_features = 4

x_trainA, y_trainA = get_data(f'{path}imagenetXtrain.npy',
                              f'{path}imagenetYtrain.npy',
                              n_features) 

x_testA, y_testA = get_data(f'{path}imagenetXtest.npy',
                            f'{path}imagenetYtest.npy',
                            n_features)

x_trainB, y_trainB = get_data(f'{path}awaXtrain.npy',
                              f'{path}awaYtrain.npy',
                              n_features)

x_testB, y_testB = get_data(f'{path}awaXtest.npy',
                            f'{path}awaYtest.npy',
                            n_features)

print('x_trainA.shape:', x_trainA.shape, 'y_trainA.shape:', y_trainA.shape)
print('x_trainB.shape:', x_trainB.shape, 'y_trainB.shape:', y_trainB.shape)
print('x_testA.shape:', x_testA.shape, 'y_testA.shape:', y_testA.shape)
print('x_testB.shape:', x_testB.shape, 'y_testB.shape:', y_testB.shape)

x_trainA.shape: (1500, 64, 64, 3) y_trainA.shape: (1500, 4)
x_trainB.shape: (1500, 64, 64, 3) y_trainB.shape: (50, 4)
x_testA.shape: (500, 64, 64, 3) y_testA.shape: (500, 4)
x_testB.shape: (500, 64, 64, 3) y_testB.shape: (500, 4)


Les deux jeux de données contiennent 2000 images, de 64x64x3 pixels, réparties en quatre classes. 1500 images sont utilisées pour l'entrainement, 500 pour l'évaluation. Seul le jeu de données A est complètement annoté, le jeu de données B ne contient que très peu d'annotations, correspondant aux 50 premiers exemples de x_trainB.

## Partie 1 : Modèle convolutionnel pour la classification d'images

On définit ci-dessous une architecture convolutionnelle simple, à ne pas modifier. Cette partie vise à évaluer la performance d'un modèle CNN sur les deux dataset A et B. Entrainez ce modèle des différentes manières possibles et reportez les performances obtenues dans le tableau situé en fin de cette partie. A chaque nouvel apprentissage, utilisez les configurations suivantes : 
- 30 epochs
- Taille des minibatchs à 64
- 10% des données pour validation

Tracez les courbes d'apprentissage, et commentez vos résultats.

In [0]:
IMG_ROWS, IMG_COLS, IMG_CHANNELS = 64, 64, 3
IMG_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)

N_EPOCHS = 30
BATCH_SIZE = 64
VAL_SPLIT = 0.1

OPTIMIZER = tf.optimizers.Adam(1e-4)

KNOWN_IMG = y_trainB.shape[0]

In [0]:
# Couches d'apprentissage de représentations par convolutions
# Renvoie un vecteur de (4096) pour une image d'entrée (64, 64, 3)
def conv_model(img_shape):
    return tf.keras.Sequential([
        # block 1
        layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same',
                    input_shape=img_shape),
        layers.BatchNormalization(axis=-1),
        layers.Activation('relu'),

        # block 2
        layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        layers.BatchNormalization(axis=-1),
        layers.Activation('relu'),

        # block 3
        layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        layers.BatchNormalization(axis=-1),
        layers.Activation('relu'),
        layers.AveragePooling2D((4, 4)),

        # block 4
        layers.Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same'),
        layers.BatchNormalization(axis=-1),
        layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        layers.AveragePooling2D((2, 2)),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Flatten()
    ])

# modèle de classification
# prend en entrée un vecteur (4096) et prédit un vecteur (4)
def classif_model(input_shape, n_features):
    return tf.keras.Sequential([
        layers.Dense(1024, activation='relu', input_shape=input_shape),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dense(n_features, activation='softmax'),
    ])

# modèle CNN
# combinaison des deux modèles ci-dessus
def cnn_model(conv_model, classif_model):
    return tf.keras.Sequential([
        conv_model,
        classif_model
    ])

### Entrainement du modèle sur les données A, puis évaluation sur A et B

In [6]:
# create models
conv_model1 = conv_model(IMG_SHAPE)
classif_model1 = classif_model(conv_model1.output.shape, n_features)
cnn_model1 = cnn_model(conv_model1, classif_model1)

# compile cnn model
cnn_model1.compile(loss='categorical_crossentropy',
                  optimizer=OPTIMIZER,
                  metrics=['accuracy'])
# display our model
print(cnn_model1.summary())

# train it 
cnn_model1.fit(x=x_trainA, y=y_trainA, validation_split=VAL_SPLIT,
               batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=1, shuffle=False)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 4096)              1110784   
_________________________________________________________________
sequential_1 (Sequential)    multiple                  4722180   
Total params: 5,832,964
Trainable params: 5,831,812
Non-trainable params: 1,152
_________________________________________________________________
None
Train on 1350 samples, validate on 150 samples
Epoch 1/30
1350/1350 [==============================] - 211s 157ms/sample - loss: 1.2642 - accuracy: 0.4370 - val_loss: 1.3880 - val_accuracy: 0.2400
Epoch 2/30
1350/1350 [==============================] - 203s 150ms/sample - loss: 0.9779 - accuracy: 0.5689 - val_loss: 1.3902 - val_accuracy: 0.2400
Epoch 3/30
1350/1350 [==============================] - 197s 146ms/sample - loss: 0.8648 - accuracy: 0.6281 - val_loss: 1.4109 - val_accuracy: 0.2200


In [7]:
# evaluate on datasets A and B
results1A = cnn_model1.evaluate(x=x_testA, y=y_testA, batch_size=BATCH_SIZE)
results1B = cnn_model1.evaluate(x=x_testB, y=y_testB, batch_size=BATCH_SIZE)
print(f'testA loss, testA acc {results1A}')
print(f'testB loss, testB acc {results1B}')

500/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

### Commentaires

* Sans surprise, notre modèle fonctionne mieux sur le dataset A que B.
* Notre apprentissage n'est pas si mauvais pour le dataset B (on fait ici deux fois mieux que le hasard - rappelons qu'il y a 4 classes à prédire)

### Entrainement et évaluation du modèle sur les données B

Attention à ne pas réutiliser la variable `model`, déjà  entrainée sur A, il faut créer une nouvelle instance `model2` à partir des fonctions **conv_model()** et **classif_model()** définies précédemment.

In [8]:
# create models
conv_model2 = conv_model(IMG_SHAPE)
classif_model2 = classif_model(conv_model2.output.shape, n_features)
cnn_model2 = cnn_model(conv_model2, classif_model2)

# compile cnn model
cnn_model2.compile(loss='categorical_crossentropy',
                  optimizer=OPTIMIZER,
                  metrics=['accuracy'])
# display our model
print(cnn_model2.summary())

# train it 
cnn_model2.fit(x=x_trainB[:KNOWN_IMG], y=y_trainB, validation_split=VAL_SPLIT,
               batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=1, shuffle=False)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 4096)              1110784   
_________________________________________________________________
sequential_4 (Sequential)    multiple                  4722180   
Total params: 5,832,964
Trainable params: 5,831,812
Non-trainable params: 1,152
_________________________________________________________________
None
Train on 45 samples, validate on 5 samples
Epoch 1/30
45/45 [==============================] - 7s 158ms/sample - loss: 1.5456 - accuracy: 0.2444 - val_loss: 1.3709 - val_accuracy: 0.4000
Epoch 2/30
45/45 [==============================] - 6s 137ms/sample - loss: 0.9812 - accuracy: 0.7333 - val_loss: 1.3667 - val_accuracy: 0.4000
Epoch 3/30
45/45 [==============================] - 6s 137ms/sample - loss: 0.7592 - accuracy: 0.6889 - val_loss: 1.3593 - val_accuracy: 0.2000
Epoch 4/30
45/45 [====

In [9]:
# evaluate on datasets A and B
results2A = cnn_model2.evaluate(x=x_testA, y=y_testA, batch_size=BATCH_SIZE)
results2B = cnn_model2.evaluate(x=x_testB, y=y_testB, batch_size=BATCH_SIZE)
print(f'testA loss, testA acc {results2A}')
print(f'testB loss, testB acc {results2B}')

500/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

### Commentaires

* C'est une véritable catastrophe, notre modèle est en fait un oracle de très mauvaise qualité - il fait aussi bien qu'un tirage aléatoire selon une distribution uniforme...
* Cette observation vaut à la fois pour le dataset A et B.

### Finetuning sur B du modèle entrainé sur A

In [10]:
# get the trained model on dataset A
cnn_model3 = cnn_model1

# freeze the conv model
cnn_model3.layers[0].trainable = False
# make sur the classification one is trainable
cnn_model3.layers[1].trainable = True

# compile cnn model
cnn_model3.compile(loss='categorical_crossentropy',
                  optimizer=OPTIMIZER,
                  metrics=['accuracy'])

# display it
print(cnn_model3.summary())

# train the classification block again on dataset B
cnn_model3.fit(x=x_trainB[:KNOWN_IMG], y=y_trainB, validation_split=VAL_SPLIT,
               batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=1, shuffle=False)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 4096)              1110784   
_________________________________________________________________
sequential_1 (Sequential)    multiple                  4722180   
Total params: 5,832,964
Trainable params: 4,722,180
Non-trainable params: 1,110,784
_________________________________________________________________
None
Train on 45 samples, validate on 5 samples
Epoch 1/30
45/45 [==============================] - 2s 49ms/sample - loss: 1.3680 - accuracy: 0.6222 - val_loss: 1.7776 - val_accuracy: 0.6000
Epoch 2/30
45/45 [==============================] - 2s 39ms/sample - loss: 1.3972 - accuracy: 0.6000 - val_loss: 1.6572 - val_accuracy: 0.6000
Epoch 3/30
45/45 [==============================] - 2s 40ms/sample - loss: 1.3019 - accuracy: 0.6444 - val_loss: 1.5254 - val_accuracy: 0.6000
Epoch 4/30
45/45 [===

In [11]:
# evaluate on datasets A and B
results3A = cnn_model3.evaluate(x=x_testA, y=y_testA, batch_size=BATCH_SIZE)
results3B = cnn_model3.evaluate(x=x_testB, y=y_testB, batch_size=BATCH_SIZE)
print(f'testA loss, testA acc {results3A}')
print(f'testB loss, testB acc {results3B}')

500/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

### Commentaires

* Victoire ! Notre tâche étant d'améliorer le score test sur le dataset B, nous pouvons considérer cette troisième méthode comme un succès.
* A noter que notre troisième classifieur fait moins bien sur A que le premier, ce qui en soit était prévisible.

* **REMARQUE IMPORTANTE : tous les résultats ont été obtenus en fixant le paramètre `shuffle=True` afin que vous puissez les reproduire à l'identique. Il est possible d'obtenir de meilleurs (ou moins boins) résultats selon le tirage vu la petitesse du jeu de données étudié.**

| | Accuracy trainA | Accuracy valA | Accuracy testA | Accuracy trainB | Accuracy valB | Accuracy testB |
|---|---|---|---|---|---|---|
| **Entrainement sur A** | 0.9726 | 0.76 | 0.716 | | | 0.502 |
| **Entrainement sur B** | | | 0.266 | 1 | 0.2 | 0.242 |
| **Entrainement sur A <br/> + Finetuning sur B** | | | 0.644 | 1 | 0.8 | 0.562 |

### **Conclusions partie 1**

Si l'on cherche à classifier un jeu de données B pour lequels on a que très peu d'exemples, il est important de chercher un jeu de données A bien plus grand qui ressemble (ou du moins où les classes de B sont inclues dans les classes de A).

En effet, la partie 1 met en lumière que :

* Entrainer un réseau de neurones (NN) exclusivement sur un jeu de données avec 50 exemples est complètement inefficace (score de 0.242 en test)

* Entrainer un NN sur un grand jeu de données A (tout est relatif, nous avons pris 1350 images en apprentissage) pour ensuiter le ré-utiliser tel quel sur un jeu de données B (qui respectent les propriétés citées ci-dessus) fonctionne bien mieux (score de 0.502 en test).

* Les meilleurs résultats sont obtenus en utlisant la technique de transfert d'apprentissage.
    * On entraîne un NN sur le jeu de données A.
    * On récupère le NN ainsi entraîné et on entraîne uniquement son bloc de classification à nouveau. Cette technique a l'avantage de transférer l'information commune (un gorille reste un gorille et un zèbre reste un zèbre !) pour une nouvelle tâche.
    * En allant plus loin sur des jeux de données bien plus conséquents, on peut se rendre compte que ce transfert se révèle aussi très efficace pour réutiliser les blocs convolutionnels les plus profonds. On ré-entraine alors notre modèle avec 1 ou 2 blocs convolutionnels + le bloc de classification, ce qui permet de *spécifier* notre réseau pour une tâche donnée.


## Partie 2 : Adaptation de domaine par alignement des activations

Cette partie consiste à implémenter et évaluer un modèle inspiré de l'article <a href="https://arxiv.org/pdf/1607.01719.pdf">Deep CORAL: Correlation Alignment for Deep Domain Adaptation [Sun & Saenko, 2016] </a>, dont la figure suivante illustre le fonctionnement. 

<center><img src="http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/coral16.png" width="50%" /></center>

Considérez les instructions suivantes :
- Utilisez les mêmes modules de convolutions que dans la Partie 1 afin de rester comparable. Pour cela, considérez une nouvelle instance **conv_model4** comme effectué précédemment.
- Seules les couches de convolutions sont partagées.
- Seulement deux couches denses après les convolutions (ie: fc6 et fc7), dropout entre les deux.
- Version simplifiée de la *CORAL loss* : minimisation de la distance entre les dernières couches denses (et/ou maximisation de la corrélation). Utilisez l'une des couches documentées sur la page suivante : https://keras.io/layers/merge/ 
- Entrainez pendant 50 epochs, avec minibatchs = 128


In [0]:
N_EPOCHS = 50
BATCH_SIZE = 128
VAL_SPLIT = 0.1

OPTIMIZER = tf.optimizers.Adam(1e-4)

### Définition du modèle

xA et xB sont les données des deux domaines, yA est la sortie du classifieur pour les données de xA dans le domaine source, et *coral* correspond à la métrique (distance ou corrélation) entre un minibatch d'exemples xA et xB, dans les deux domaines. Le modèle optimise les deux loss : *categorical_crossentropy* pour classer les images du domaine source; et *mse* qui minimise (ou maximise) la métrique considérée.


In [17]:
x1 = layers.Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))
x2 = layers.Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))

conv1, conv2 = conv_model1(x1), conv_model1(x2)

def correlation_alignment_deep_domain_adaptation(conv1, conv2, n_classes):
    y1 = layers.Dense(1024, activation='relu')(conv1)
    y1 = layers.Dropout(0.5)(y1)
    y1 = layers.Dense(512, activation='relu')(y1)

    y2 = layers.Dense(1024, activation='relu')(conv2)
    y2 = layers.Dropout(0.5)(y2)
    y2 = layers.Dense(512, activation='relu')(y2)

    dot = layers.dot([y1, y2], axes=-1, normalize=True)
    y = layers.Dense(n_classes, activation='softmax')(y1)

    return dot, y

y, dot = correlation_alignment_deep_domain_adaptation(conv1, conv2, n_features)

model4 = tf.keras.Model([x1, x2], [y, dot])
model4.compile(loss=['categorical_crossentropy', 'mse'],
               optimizer=OPTIMIZER,
               metrics=['accuracy'])
print(model4.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 4096)         1110784     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 1024)         4195328     sequential[6][0]             

### Entrainement du modèle d'adaptation par alignement

Selon le critère *coral* que vous optimisez, considérez le vecteur objectif ne contenant que des 1 ou que des 0..

In [18]:
onesA = np.ones((len(x_trainA), 1))
onesB = np.ones((len(x_trainB), 1))

# train model
model4.fit([x_trainA, x_trainB], [y_trainA, onesA],
           epochs=N_EPOCHS, 
           batch_size=BATCH_SIZE,
           validation_split=VAL_SPLIT)

# eval on A and B
results4A = model4.evaluate([x_testA, x_testB], [y_trainA, onesA])
results4B = model4.evaluate([x_testB, x_testB], [y_trainB, onesB])

print(f'testA loss, testA acc {results4A}')
print(f'testB loss, testB acc {results4B}')

ValueError: ignored

### **Conclusions partie 2**

* 

*

## Partie 3 : Adaptation de domaine par contrainte adversarial

Cette partie consiste à implémenter un modèle d'adaptation de modèle par contrainte adversarial, inspiré de l'article <a href="http://openaccess.thecvf.com/content_cvpr_2017/papers/Tzeng_Adversarial_Discriminative_Domain_CVPR_2017_paper.pdf">Adversarial Discriminative Domain Adaptation [Tzeng, 2017]</a>.

<center><img src="http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/adversadapt17.png" width="80%" /></center>

Dans la figure ci-dessus, les modules en pointillés indiquent une réutilisation de modèles déjà  entrainés, tandis que les modules en lignes pleines désignent des modèles nouvellement entrainés. Pour la phase de *pre-training*, considérez le modèle entrainé sur le dataset A dans la partie 1. 

La stratégie consiste donc à :
- Faire le pré-training du modèle source sur les données source avec un critère de classification. Cette étape a été faite en partie 1 : **conv_model** et **classif_model** sont déjà entrainés.
- Ne pas ré-entrainer ces deux modules.
- Apprendre from scratch le modèle extracteur de caractéristiques pour les données cible à  l'aide d'un discriminateur adversarial appris à  discriminer entre les données source transformées par le modèle pré-appris et les données du domaine cible transformées par le modèle cible.  
- Construire un classifieur en empilant la couche de classification du modèle source sur l'extracteur de caractéristiques sur les données cible.



## Définition du modèle discriminateur
Le discriminateur a pour objectif de distinguer les domaines de deux sources de données (A et B) à partir de représentations extraites de couches de convolutions. Ecrivez un modèle simple à deux couches cachées. 

In [0]:
classif_model5 = classif_model(conv_model1.output.shape, n_features)
classif_model5.compile(loss='categorical_crossentropy',
                       optimizer=OPTIMIZER,
                       metrics=['accuracy'])

print(classif_model5.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, None, 1024)        4195328   
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 1024)        0         
_________________________________________________________________
dense_13 (Dense)             (None, None, 512)         524800    
_________________________________________________________________
dense_14 (Dense)             (None, None, 4)           2052      
Total params: 4,722,180
Trainable params: 4,722,180
Non-trainable params: 0
_________________________________________________________________
None


### Modèle adversarial

Ecrivez ci-dessous le modèle adversarial qui combine le module *Target CNN* et le discriminateur. *Target CNN* consistera en une nouvelle instance **conv_model5** pour extraire des représentations similaires au modèle entrainé en Partie 1 sur les images sources.

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def gan_loss(logits, is_real=True):
    """
    Computes standard gan loss between logits and labels
    """
    labels = tf.ones_like(logits) if is_real else tf.zeros_like(logits)
    return cross_entropy(labels, logits)

class GAN(tf.keras.Model):
    """
    A basic GAN class based on tf.keras.Model
    """

    def __init__(self, **kwargs):
        super(GAN, self).__init__()
        self.__dict__.update(kwargs)

    def generate(self, noise, training):
        return self.G(noise, training)

    def discriminate(self, x, training):
        return self.D(x, training)

    # `tf.function` annotation causes the function to be "compiled".
    @tf.function
    def train(self, real_x, real_z):
        """
        Passes through the network, computes loss, compute gradients then update
        the network.
        """
        # copied from tp3, just changing the input noise
        noise = real_z

        with tf.GradientTape() as G_tape, tf.GradientTape() as D_tape:
            # run noise through generator
            fake_x = self.generate(noise, training=True)

            # discriminate real_x and fake_x
            real_output = self.discriminate(real_x, training=True)
            fake_output = self.discriminate(fake_x, training=True)

            # compute real and fake losses for discriminator
            D_real_loss = gan_loss(logits=real_output, is_real=True)
            D_fake_loss = gan_loss(logits=fake_output, is_real=False)
            D_loss = D_fake_loss + D_real_loss

            # compute generator loss
            G_loss = gan_loss(logits=fake_output, is_real=True)
        
        # compute gradients
        G_gradients = G_tape.gradient(G_loss, self.G.trainable_variables)
        D_gradients = D_tape.gradient(D_loss, self.D.trainable_variables)

        # update network weights
        self.G_optimizer.apply_gradients(
            zip(G_gradients, self.G.trainable_variables)
        )
        self.D_optimizer.apply_gradients(
            zip(D_gradients, self.D.trainable_variables)
        )

        return D_real_loss, D_fake_loss, D_loss, G_loss

### Modèle utilisé en phase de test

Construisez ci-dessous le modèle final qui combine les modules *Target CNN*  (**conv_model4**) entrainés précédemment et **classif_model** entrainé en Partie 1.

In [0]:
# todo...

### Entrainement du modèle adversarial

Utilisez les fonctions de création de minibatchs suivantes qui randomisent les ensembles d'apprentissage à chaque nouvelle époque.


In [0]:
batch_size = 64
nbdata = len(x_trainA)

def get_batchA():
    global x_trainA, y_trainA
    i = 0
    while True:
        i = i + batch_size
        if i+batch_size > nbdata:
            i = 0
            lidx = list(range(nbdata))
            shuffle(lidx)
            x_trainA = x_trainA[lidx]
            y_trainA = y_trainA[lidx]
        yield x_trainA[i:i+batch_size], y_trainA[i:i+batch_size]

def get_batchB():
    global x_trainB, y_trainB
    i = 0
    while True:
        i = i + batch_size
        if i+batch_size > nbdata:
            i = 0
            lidx = list(range(nbdata))
            shuffle(lidx)
            x_trainB = x_trainB[lidx]
        yield x_trainB[i:i+batch_size]

data_genA = get_batchA()
data_genB = get_batchB()

<h4> Boucle d'apprentissage </h4>

In [0]:
ones = np.ones((batch_size,1))
zeros = np.zeros((batch_size,1))
nb_batchs = int(len(x_trainA)/batch_size)

for epoch in range(30):
    loss = 0.0
    acc = 0.0
    for batch in range(nb_batchs):
        # get minibatchs
        xA, yA = next(data_genA)
        xB = next(data_genB)

        # A complÃ©ter
        
        # train discriminator
        
        # train adversarial model on new minibatch
        
        # monitoring

Commentaires :

<h4> Evaluation sur les dataset A et B </h4>

In [0]:
# todo...

Commentaires :

<hr/>

**Rappel, pour l'anonymisation de votre rendu, veuillez suivre les consignes suivantes :**

- Choisissez un numéro aléatoire à 6 chiffres, le plus aléatoire possible..
- Ecrivez ce numéro sur une feuille, ainsi que votre nom et numéro d'étudiant 
- Pliez cette feuille en 4, et donnez la à la fin de l'examen au moment d'émarger  
- Zippez votre fichier notebook, et nommez l'archive avec votre numéro aléatoire 
- Envoyez l'archive via la page : http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/upload/upload.php
